In [0]:
from pyspark.sql.functions import *


In [0]:
data=spark.read.format("csv").option('header','true').option('inferschema','true') \
    .load("abfss://bronze@pronetflixdata.dfs.core.windows.net/titles");


In [0]:
#data.display()


# handling the null values


In [0]:
data.printSchema()

# here i need to change the schema first before making any transformations.


In [0]:
from pyspark.sql.types import *

In [0]:
given_schema = StructType([
    StructField("duration_minutes", IntegerType(), True),
    StructField("duration_seasons", IntegerType(), True),
    StructField("type", StringType(), True),
    StructField("title", StringType(), True),
    StructField("date_added", StringType(), True),
    StructField("release_year", IntegerType(), True),
    StructField("rating", StringType(), True),
    StructField("description", StringType(), True),
    StructField("show_id", IntegerType(), True)
])

In [0]:
data2=spark.read.format("csv").option('header','true').schema(given_schema) \
    .load("abfss://bronze@pronetflixdata.dfs.core.windows.net/titles");


In [0]:
data2.printSchema()

# filling Nulls 

In [0]:
#data2=data2.withColumn('duration_minutes', col('duration_minutes').cast('integer') );

data2=data2.fillna({'duration_minutes': 0,'duration_seasons': 1})


In [0]:
#data2.display()

# make a short_title from title column

In [0]:
data2=data2.withColumn( 'short_title', split(col('title'), ':')[0] );
data2.display()


# Add the flag column also

In [0]:
data2=data2.withColumn('flag', when(col('type')=='Movie', 1).\
                        when(col('type')=='TV Show',2). \
                          otherwise(0) );
data2.display()

In [0]:
data2.write.format('delta').mode('append').save("abfss://silver@pronetflixdata.dfs.core.windows.net/netflix_titles")